# Mavlink Camera
> Mavlink Camera Component for sending commands to a camera on a companion computer or GCS
> The server is on the companion computer and the client is on the ground station PC.

In [ ]:
#| default_exp mavlink.camera

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:
#| hide
# skip_showdoc: true to avoid running cells when rendering docs, and 
# skip_exec: true to skip this notebook when running tests. 
# this should be a raw cell 

In [ ]:
#| export
import time, os, sys

from UAV.camera.fake_cam import GSTCamera, BaseCamera
from UAV.logging import logging
from UAV.mavlink.mavcom import MAVCom, time_since_boot_ms, time_UTC_usec, boot_time_str, date_time_str
from UAV.mavlink.component import Component, mavutil, mavlink, MAVLink
import threading
import cv2
import numpy as np
import toml
# try:
#     # https://hackernoon.com/how-to-manage-configurations-easily-using-toml-files
#     import tomllib   # Python 3.11+
# except ModuleNotFoundError:
#     import tomli as tomllib
# from UAV.imports import *   # TODO why is this relative import on nbdev_export?


In [ ]:
#| hide
from fastcore.utils import *
from nbdev.showdoc import *
from fastcore.test import *

### Implementation of these commands:
>
> [MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS = 527](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS)
[MAV_CMD_REQUEST_CAMERA_INFORMATION = 523](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION)
[MAV_CMD_REQUEST_CAMERA_SETTINGS = 524](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS)
[MAV_CMD_REQUEST_STORAGE_INFORMATION = 525](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION)
[MAV_CMD_STORAGE_FORMAT = 526](https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT)
[MAV_CMD_SET_CAMERA_ZOOM = 531](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM)
[MAV_CMD_SET_CAMERA_FOCUS = 532](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_FOCUS)
[MAV_CMD_IMAGE_START_CAPTURE = 2000](https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE)
[MAV_CMD_IMAGE_STOP_CAPTURE = 2001](https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE)
> 
> [MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION = 2504](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION)
[MAV_CMD_REQUEST_VIDEO_STREAM_STATUS = 2505](https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_STATUS)
[MAV_CMD_VIDEO_START_CAPTURE = 2500](https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE)
[MAV_CMD_VIDEO_STOP_CAPTURE = 2501](https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE)
[MAV_CMD_SET_CAMERA_MODE = 530](https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE)
> 
**Note**
The simulated camera is implemented in PX4 [gazebo_camera_manager_plugin.cpp](https://github.com/PX4/PX4-SITL_gazebo-classic/blob/main/src/gazebo_camera_manager_plugin.cpp).


In [ ]:
#| export
# from pymavlink.dialects.v20 import ardupilotmega as mav
# from pymavlink.dialects.v20.ardupilotmega import MAVLink


NAN = float("nan")

"""
MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS = 527 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS
MAV_CMD_REQUEST_CAMERA_INFORMATION = 521 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
MAV_CMD_REQUEST_CAMERA_SETTINGS = 522 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
MAV_CMD_REQUEST_STORAGE_INFORMATION = 525 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
MAV_CMD_STORAGE_FORMAT = 526 # https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT
MAV_CMD_SET_CAMERA_ZOOM = 531 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
MAV_CMD_SET_CAMERA_FOCUS = 532 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_FOCUS
MAV_CMD_IMAGE_START_CAPTURE = 2000  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
MAV_CMD_IMAGE_STOP_CAPTURE = 2001  # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE
MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION = 2504 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_INFORMATION
MAV_CMD_REQUEST_VIDEO_STREAM_STATUS = 2505 # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_VIDEO_STREAM_STATUS
MAV_CMD_VIDEO_START_CAPTURE = 2500 # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE
MAV_CMD_VIDEO_STOP_CAPTURE = 2501 # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE
MAV_CMD_SET_CAMERA_MODE = 530 # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE

"""
CAMERA_INFORMATION = mavlink.MAVLINK_MSG_ID_CAMERA_INFORMATION # https://mavlink.io/en/messages/common.html#CAMERA_INFORMATION
CAMERA_SETTINGS = mavlink.MAVLINK_MSG_ID_CAMERA_SETTINGS # https://mavlink.io/en/messages/common.html#CAMERA_SETTINGS
STORAGE_INFORMATION = mavlink.MAVLINK_MSG_ID_STORAGE_INFORMATION # https://mavlink.io/en/messages/common.html#STORAGE_INFORMATION
CAMERA_CAPTURE_STATUS = mavlink.MAVLINK_MSG_ID_CAMERA_CAPTURE_STATUS # https://mavlink.io/en/messages/common.html#CAMERA_CAPTURE_STATUS
CAMERA_IMAGE_CAPTURED = mavlink.MAVLINK_MSG_ID_CAMERA_IMAGE_CAPTURED # https://mavlink.io/en/messages/common.html#CAMERA_IMAGE_CAPTURED


In [ ]:
#| export








def read_camera_info_from_toml(toml_file_path):
    """Read MAVLink camera info from a TOML file."""
    with open(toml_file_path, 'rb') as file:
        data = toml.load(file)

    camera_info = data['camera_info']
    camera_info['vendor_name'] = [int(b) for b in camera_info['vendor_name'].encode()]
    camera_info['model_name'] = [int(b) for b in camera_info['model_name'].encode()]
    # Extract camera_info
    return data['camera_info']

class WaitMessage:
    """Wait for a specific message from the server"""
    def __init__(self, target_system, target_component):
        # self.target_system = None
        # self.target_component = None
        self._condition = None
        self._event = threading.Event()
        self._object = None
        self._log = logging.getLogger("uav.{}".format(self.__class__.__name__))
        # self._msg_id = None

    @property
    def log(self) -> logging.Logger:
        return self._log

    def set_condition(self, msg_id, target_system, target_component):
        """Set the condition function to generate event for a specific message received from the server"""
        assert target_system is not None and target_component is not None, "call set_target(target_system, target_component) first"
        # self._msg_id = msg_id
        # self.target_system = target_system
        # self.target_component = target_component
        self._condition = (lambda msg: msg.get_msgId() == msg_id
                                   and msg.get_srcSystem() == target_system
                                   and msg.get_srcComponent() == target_component)
        self._event.clear()

    def on_condition(self, msg):
        """Event for a specific received from the server"""
        if self._condition is not None:
            # self.log.debug(f"!!!! on_condition : {msg.get_msgId() = } {self._msg_id = } ")
            # self.log.debug(f"!{msg.get_msgId() == self._msg_id} and {msg.get_srcSystem() = } {self.target_system} and {msg.get_srcComponent() == self.target_component } ")
            # self.log.debug(f"!!!!  {self._condition(msg) = } ")
            if self._condition(msg):
                # self.log.debug(f"!!!! on_condition meet : {msg = } ")
                self._object = msg
                self._event.set()
        # except:
        #     pass


    def set(self, value):
        """Set the object and set the event."""
        self._object = value
        self._event.set()

    def clear(self):
        """Clear the object and clear the event."""
        self._object = None
        self._event.clear()

    def get(self, timeout=1):
        """Get the object if the event is set or wait until it's set with an optional timeout.

        Returns:
            The object if the event is set, or None if it times out or the event isn't set.
        """
        is_set = self._event.wait(timeout)
        if is_set:
            return self._object
        return None

    def is_set(self):
        """Check if the event is set."""
        return self._event.is_set()


In [ ]:
show_doc(WaitMessage)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#L68){target="_blank" style="float:right; font-size:smaller"}

### WaitMessage

>      WaitMessage (target_system, target_component)

Wait for a specific message from the server

In [ ]:
show_doc(WaitMessage.set_condition)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#L83){target="_blank" style="float:right; font-size:smaller"}

### WaitMessage.set_condition

>      WaitMessage.set_condition (msg_id, target_system, target_component)

Set the condition function to generate event for a specific message received from the server

In [ ]:
show_doc(WaitMessage.get)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#L118){target="_blank" style="float:right; font-size:smaller"}

### WaitMessage.get

>      WaitMessage.get (timeout=1)

Get the object if the event is set or wait until it's set with an optional timeout.

Returns:
    The object if the event is set, or None if it times out or the event isn't set.

In [ ]:
#| export

class CameraClient(Component):
    """Create a Viewsheen mavlink gimbal client component for send commands to a gimbal on a companion computer or GCS """

    def __init__(self,
                 source_component,  # used for component indication
                 mav_type,  # used for heartbeat MAV_TYPE indication
                 debug):  # logging level
        
        super().__init__(source_component=source_component, mav_type=mav_type, debug=debug)
        self.set_message_callback(self.on_message)
        self.wait_for_message = WaitMessage(self.target_system, self.target_component)

    def on_mav_connection(self):
        pass

    def on_message(self, msg):
        """Callback for a command received from the server"""
        print(f"CAMERA_Client  {msg} ")
        self.wait_for_message.on_condition(msg)
        # if msg.get_type() == "CAMERA_IMAGE_CAPTURED":
        #     # print(f"Camera Capture Status {msg = }")
        #     print(msg)
        #         # f"{msg.image_count = }  {msg.image_interval = } {msg.recording_time_ms = } {msg.available_capacity = } {msg.image_status = } {msg.video_status = } {msg.image_interval = } ")
        #
        #     return True
            # print(f"Camera Image Captured {msg = }")


    def send_message(self, msg):
        """Send a message to the camera"""
        self.master.mav.send(msg)
        self.log.debug(f"Sent {msg}")

    def request_camera_capture_status(self):
        """Request camera capture status"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS
        # see https://github.com/PX4/PX4-SITL_gazebo-classic/blob/main/src/gazebo_camera_manager_plugin.cpp#L543

        assert self.target_system is not None and self.target_component is not None, "call set_target(target_system, target_component) first"
        self.wait_for_message.set_condition(CAMERA_CAPTURE_STATUS, self.target_system, self.target_component)
        t = self.send_command(  self.target_system,
                                self.target_component,
                                mavlink.MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS,
                                [1,0,0,0,0,0,0]
                              )
        return self.wait_for_message.get()


    def request_camera_information(self):
        """Request camera information"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
        assert self.target_system is not None and self.target_component is not None, "call set_target(target_system, target_component) first"
        self.wait_for_message.set_condition(CAMERA_INFORMATION, self.target_system, self.target_component)
        t = self.send_command(  self.target_system,
                                self.target_component,
                                mavlink.MAV_CMD_REQUEST_CAMERA_INFORMATION,
                                [1,0,0,0,0,0,0]
                              )
        return self.wait_for_message.get()

    def request_camera_settings(self):
        """Request camera settings"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
        self.wait_for_message.set_condition(CAMERA_SETTINGS, self.target_system, self.target_component)
        t = self.send_command(  self.target_system,
                                self.target_component,
                                mavlink.MAV_CMD_REQUEST_CAMERA_SETTINGS,
                                [0,0,0,0,0,0,0]
                              )

        return self.wait_for_message.get()

    def request_storage_information(self):
        """Request storage information (for cases where camera has storage)"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
        self.wait_for_message.set_condition(STORAGE_INFORMATION, self.target_system, self.target_component)
        t = self.send_command(  self.target_system,
                                self.target_component,
                                mavlink.MAV_CMD_REQUEST_STORAGE_INFORMATION,
                                [0,0,0,0,0,0,0]
                              )

        return self.wait_for_message.get()

    def storage_format(self):
        """Format storage (for cases where camera has storage)"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT
        self.wait_for_message.set_condition(STORAGE_INFORMATION, self.target_system, self.target_component)
        t = self.send_command(  self.target_system,
                                self.target_component,
                                mavlink.MAV_CMD_STORAGE_FORMAT,
                                [0,0,0,0,0,0,0]
                              )
        return self.wait_for_message.get()


    def set_camera_mode(self, mode_id): # https://mavlink.io/en/messages/common.html#CAMERA_MODE
        """ Set the camera mode"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE
        # https://mavlink.io/en/messages/common.html#CAMERA_MODE

        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_SET_CAMERA_MODE,
                                [0,
                                 mode_id, # https://mavlink.io/en/messages/common.html#CAMERA_MODE
                                 0,0,0,0,0]
                              )


    def set_camera_zoom(self, value): # 0 to 100 zoom value
        """ Set the camera zoom"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
        self.wait_for_message.set_condition(CAMERA_SETTINGS, self.target_system, self.target_component)
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_SET_CAMERA_ZOOM,
                                [0,  # https://mavlink.io/en/messages/common.html#CAMERA_ZOOM_TYPE
                                 value, 0,0,0,0,0]
                              )

        return self.wait_for_message.get()


    def image_start_capture(self, interval=0, # Image capture interval
                            count=1, # Number of images to capture (0 for unlimited)
                            ):
        """Start image capture sequence."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
        try:
            self.image_capture_sequence_number += 1
            if count != 1:
                self.image_capture_sequence_number = 0
        except:
            self.image_capture_sequence_number = 1

        self.wait_for_message.set_condition(CAMERA_IMAGE_CAPTURED, self.target_system, self.target_component)
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_IMAGE_START_CAPTURE,
                                [0,
                                 interval, # interval
                                 count, # number of  images to capture
                                 self.image_capture_sequence_number, # Sequence number starting from 1. This is only valid for single-capture (param3 == 1), otherwise set to 0.  Increment the capture ID for each capture command to prevent double captures when a command is re-transmitted.
                                 NAN, # Reserved
                                 NAN, # Reserved
                                 NAN]
                              ) # Reserved
        return self.wait_for_message.get()


    def image_stop_capture(self):
        """Stop image capture sequence"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_IMAGE_STOP_CAPTURE,
                                [0, NAN, NAN, NAN, NAN, NAN, NAN]
                              )

    def video_start_capture(self, video_stream_id=0, # Video stream id (0 for all streams)
                            frequency=1): # Frequency CAMERA_CAPTURE_STATUS messages should be sent while recording (0 for no messages, otherwise frequency in Hz)
        """Start video capture"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_VIDEO_START_CAPTURE,
                                [video_stream_id, # Video stream id (0 for all streams)
                                 frequency, # Frequency CAMERA_CAPTURE_STATUS messages should be sent while recording (0 for no messages, otherwise frequency in Hz)
                                 NAN, NAN, 0, 0, NAN]
                              )

    def video_stop_capture(self, video_stream_id=0): # Video stream id (0 for all streams)
        """Stop video capture"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_VIDEO_STOP_CAPTURE,
                                [video_stream_id, NAN, NAN, NAN, NAN, NAN, NAN]
                              )

    def video_start_streaming(self, video_stream_id=0, # Video Stream ID (0 for all streams)
                                ):
        """Start video streaming"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_STREAMING
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_VIDEO_START_STREAMING,
                                [video_stream_id, NAN, NAN, NAN, NAN, NAN, NAN]
                                )

    def video_stop_streaming(self, video_stream_id=0): # Video Stream ID (0 for all streams)
        """Stop the video stream"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_STREAMING
        t = self.send_command(self.target_system, self.target_component,
                                mavlink.MAV_CMD_VIDEO_STOP_STREAMING,
                                [video_stream_id, NAN, NAN, NAN, NAN, NAN, NAN]
                              )



In [ ]:
show_doc(CameraClient)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#L135){target="_blank" style="float:right; font-size:smaller"}

### CameraClient

>      CameraClient (source_component, mav_type, debug)

Create a Viewsheen mavlink gimbal client component for send commands to a gimbal on a companion computer or GCS

|    | **Details** |
| -- | ----------- |
| source_component | used for component indication |
| mav_type | used for heartbeat MAV_TYPE indication |
| debug | logging level |

In [ ]:
#| export



# class BaseCamera:
#     def __init__(self,
#                  camera_dict=None,  # camera_info dict
#                  debug=False):  # debug log flag
#         self.mav: MAVLink = None
#         camera_dict = {'camera_info':{
#                 'vendor_name': 'UAV',
#                 'model_name': 'FakeCamera',
#                 'firmware_version': 1,
#                 'focal_length': 2.8,
#                 'sensor_size_h': 3.2,
#                 'sensor_size_v': 2.4,
#                 'resolution_h': 640,
#                 'resolution_v': 480,
#                 'lens_id': 0,
#                 'flags': 0,
#                 'cam_definition_version': 1,
#                 'cam_definition_uri': '',
#                 }}
#
#         self.camera_info = self.get_camera_info(camera_dict)
#
#
#     def get_camera_info(self, camera_dict):
#         """get  MAVLink camera info from a TOML dict."""
#         def make_length_32(s: str) -> str:
#             if len(s) > 32:
#                 return s[:32]
#             return s.ljust(32)  # pad with spaces to the right to make length 32
#
#         camera_info = camera_dict['camera_info']
#         # vender name and model name must be 32 bytes long
#         camera_info['vendor_name'] = make_length_32(camera_info['vendor_name'])
#         camera_info['model_name'] = make_length_32(camera_info['model_name'])
#         camera_info['vendor_name'] = [int(b) for b in camera_info['vendor_name'].encode()]
#         camera_info['model_name'] = [int(b) for b in camera_info['model_name'].encode()]
#
#         return camera_info
#
#     def camera_information_send(self):
#         """ Information about a camera. Can be requested with a
#             MAV_CMD_REQUEST_MESSAGE command."""
#         # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
#         self.mav.camera_information_send(time_since_boot_ms(),  # time_boot_ms
#                                             self.camera_info['vendor_name'],         # vendor name
#                                             self.camera_info['model_name'],          # model name
#                                             self.camera_info['firmware_version'],    # firmware version
#                                             self.camera_info['focal_length'],        # focal length
#                                             self.camera_info['sensor_size_h'],       # sensor size h
#                                             self.camera_info['sensor_size_v'],       # sensor size v
#                                             self.camera_info['resolution_h'],        # resolution h
#                                             self.camera_info['resolution_v'],        # resolution v
#                                             self.camera_info['lens_id'],             # lend_id
#                                             self.camera_info['flags'],               # flags
#                                             self.camera_info['cam_definition_version'],          # cam definition version
#                                             bytes(self.camera_info['cam_definition_uri'], 'utf-8'), # cam definition uri
#                                          )
#     def close(self) :
#         pass


class CameraServer(Component):
    """Create a mavlink Camera server Component using a test GSTREAMER pipeline"""

    def __init__(self,
                 source_component,  # used for component indication
                 mav_type,  # used for heartbeat MAV_TYPE indication
                 camera, # camera  (or FakeCamera for testing)
                 debug,  # logging level
                ):
        super().__init__( source_component=source_component, mav_type=mav_type, debug=debug)
        self.set_message_callback(self.on_message)


        self.camera:GSTCamera = camera


    def on_mav_connection(self):
        """Start the mavlink connection"""
        assert self.mav is not None, "call set_mav first"
        if self.camera is None:
            self.log.warning(f"Component has no camera object")
            self.camera = BaseCamera()
        self.camera.mav = self.mav  # set the mavlink connection for mavlink messages


    def on_message(self, msg # type: Message
                   ) -> bool: # return True to indicate that the message has been handled
        """Callback for a command received from the client
        """

        if msg.get_type() == "COMMAND_LONG":
            # print(f"Command  {msg.command = } ")
            if msg.command == mavlink.MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS:
                self.camera_capture_status_send()
                return True # return True to indicate that the message has been handled

            elif msg.command == mavlink.MAV_CMD_REQUEST_CAMERA_INFORMATION:
                self.camera_information_send()
                return True # return True to indicate that the message has been handled

            elif msg.command == mavlink.MAV_CMD_REQUEST_CAMERA_SETTINGS:
                # self.send_ack(msg)
                self.camera_settings_send()
                return True

            elif msg.command == mavlink.MAV_CMD_REQUEST_STORAGE_INFORMATION:
                self.storage_information_send()
                return True

            elif msg.command == mavlink.MAV_CMD_STORAGE_FORMAT:
                self.storage_format(msg)
                return True

            elif msg.command == mavlink.MAV_CMD_SET_CAMERA_ZOOM:
                # self.log.info(f"***** Zoom {msg}")
                # print(f"Zoom {msg.param2 = }")
                self.set_camera_zoom(msg)
                return True

            elif msg.command == mavlink.MAV_CMD_IMAGE_START_CAPTURE:
                self.image_start_capture(msg)
                return True # return True to indicate that the message has been handled

            elif msg.command == mavlink.MAV_CMD_IMAGE_STOP_CAPTURE:
                self.image_stop_capture(msg)
                return True

            elif msg.command == mavlink.MAV_CMD_VIDEO_START_CAPTURE:
                self.video_start_capture(msg)
                return True

            elif msg.command == mavlink.MAV_CMD_VIDEO_STOP_CAPTURE:
                self.video_stop_capture(msg)
                return True

            elif msg.command == mavlink.MAV_CMD_SET_CAMERA_MODE:
                self.set_camera_mode(msg)
                return True # return True to indicate that the message has been handled

            elif msg.command == mavlink.MAV_CMD_VIDEO_START_STREAMING:
                self.video_start_streaming(msg)
                return True # return True to indicate that the message has been handled

            elif msg.command == mavlink.MAV_CMD_VIDEO_STOP_STREAMING:
                self.video_stop_streaming(msg)
                return True # return True to indicate that the message has been handled


            
        else:
            self.log.debug(f"Unknown command {msg.get_type()} received from {msg.get_srcSystem()}/{msg.get_srcComponent()}")
            return False # return False to indicate that the message has not been handled

        
    def camera_capture_status_send(self):
        """ Information about the status of a capture. Can be requested with a
            MAV_CMD_REQUEST_MESSAGE command."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_CAPTURE_STATUS
        try:
            self.camera.camera_capture_status_send()
        except AttributeError as err:
            self.log.error(f"{err = }")

        # if self.camera is None:
        #     self.log.warning(f"Component has no camera object")
        #     return
        # self.mav.camera_capture_status_send(time_since_boot_ms(), # time_boot_ms
        #                                     0, # image status
        #                                     0, # video status
        #                                     0, # image interval
        #                                     0, # recording time ms
        #                                     0, # available capacity
        #                                     0, # image count
        #                                     )

    def camera_information_send(self):
        """ Information about a camera. Can be requested with a
            MAV_CMD_REQUEST_MESSAGE command."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_INFORMATION
        try:
            self.camera.camera_information_send()
        except AttributeError as err:
            self.log.error(f"{err = }")


    def camera_settings_send(self):
        """ Settings of a camera. Can be requested with a
            MAV_CMD_REQUEST_MESSAGE command."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_CAMERA_SETTINGS
        try:
            self.camera.camera_settings_send()
        except AttributeError as err:
            self.log.error(f"{err = }")

        # if self.camera is None:
        #     self.log.warning(f"Component has no camera object")
        #     return
        # nan = float("nan")
        # self.mav.camera_settings_send(time_since_boot_ms(), # time_boot_ms
        #                               0, # https://mavlink.io/en/messages/common.html#CAMERA_MODE
        #                               NAN, # Current zoom level as a percentage of the full range (0.0 to 100.0, NaN if not known)
        #                               NAN, # Current focus level as a percentage of the full range (0.0 to 100.0, NaN if not known)
        #                               )

    def storage_information_send(self):
        """ Information about a storage medium. This message is sent in response to
            MAV_CMD_REQUEST_MESSAGE."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_REQUEST_STORAGE_INFORMATION
        try:
            self.camera.storage_information_send()
        except AttributeError as err:
            self.log.error(f"{err = }")



    def storage_format(self, msg):
        """ A message containing the result of the format attempt (asynchronous)."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_STORAGE_FORMAT
        self.log.error(f"storage_format_send not implemented")
        # do format storage
        storage_id = msg.param1
        format =    msg.param2
        reset_image_log = msg.param3
        self.storage_information_send()
 



    def set_camera_zoom(self, msg):
        """ Set the camera zoom """
        # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_ZOOM
        try:
            self.camera.set_camera_zoom(msg.param2)
        except AttributeError as err:
            self.log.error(f"{err = }")
        
        
    def image_start_capture(self, msg):
        """Start image capture sequence."""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_START_CAPTURE
        try:
            interval = msg.param2
            count = msg.param3
            self.camera.image_start_capture(interval, count)
        except AttributeError as err:
            self.log.error(f"{err = }")


    def image_stop_capture(self, msg):
        """Stop image capture sequence"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_IMAGE_STOP_CAPTURE
        try:

            self.camera.image_stop_capture()
        except AttributeError as err:
            self.log.error(f"{err = }")


    def video_start_capture(self, msg):
        """Start video capture"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_CAPTURE
        try:
            stream_id = msg.param1
            frequency = msg.param2  # Frequency CAMERA_CAPTURE_STATUS messages should be sent while recording (0 for no messages, otherwise frequency in Hz)

            self.camera.video_start_capture(stream_id, frequency)
        except AttributeError as err:
            self.log.error(f"{err = }")



    def video_stop_capture(self, msg):
        """Stop video capture"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_CAPTURE
        try:
            self.camera.video_stop_capture()
        except AttributeError as err:
            self.log.error(f"{err = }")


    def video_start_streaming(self, msg):
        """Start video streaming"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_START_STREAMING
        # todo get parameters from message
        print("todo get parameters from message")
        try:
            self.camera.video_start_streaming(0)
        except AttributeError as err:
            self.log.error(f"{err = }")


    def video_stop_streaming(self, msg):
        """Stop video streaming"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_VIDEO_STOP_STREAMING
        # todo get parameters from message
        print("todo get parameters from message")
        try:
            self.camera.video_stop_streaming(0)
        except AttributeError as err:
            self.log.error(f"{err = }")


    def set_camera_mode(self, msg):
        """ Set the camera mode"""
        # https://mavlink.io/en/messages/common.html#MAV_CMD_SET_CAMERA_MODE
        try:
            self.camera.set_camera_mode(msg.param2)
        except AttributeError as err:
            self.log.error(f"{err = }")





    def close(self):
        """Close the connection to the camera"""
        if self.camera is not None:
            self.camera.close()
        super().close()
        self.log.debug(f"Closed connection to camera")
        return True
    


In [ ]:
show_doc(CameraServer)

---

[source](https://github.com/johnnewto/UAV/blob/main/UAV/mavlink/camera.py#L506){target="_blank" style="float:right; font-size:smaller"}

### CameraServer

>      CameraServer (source_component, mav_type, camera, debug)

Create a mavlink Camera server Component using a test GSTREAMER pipeline

|    | **Details** |
| -- | ----------- |
| source_component | used for component indication |
| mav_type | used for heartbeat MAV_TYPE indication |
| camera | camera  (or FakeCamera for testing) |
| debug | logging level |

In [ ]:
MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA
# cli = GimbalClient(mav_connection=None, source_component=11, mav_type=MAV_TYPE_GCS, debug=False)
# gim1 = GimbalServer(mav_connection=None, source_component=22, mav_type=MAV_TYPE_CAMERA, debug=False)

con1, con2 = "udpin:localhost:14445", "udpout:localhost:14445"
# con1, con2 = "/dev/ttyACM0", "/dev/ttyUSB0"
# with MAVCom(con1, source_system=111, debug=False) as client:
#     with MAVCom(con2, source_system=222, debug=False) as server:
#         cam:CameraClient = client.add_component(CameraClient( mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
#         server.add_component(CameraServer( mav_type=MAV_TYPE_CAMERA, source_component = 22, camera=None, debug=False))
#         
#         cam.wait_heartbeat(target_system=222, target_component=22, timeout=1)
#         time.sleep(0.1)
#         cam.set_target(222, 22)
with MAVCom(con1, source_system=111, debug=False) as client:
    with MAVCom(con2, source_system=222, debug=False) as server:
        cam:CameraClient = client.add_component(
            CameraClient(mav_type=MAV_TYPE_GCS, source_component=11, debug=True))
        # server.add_component(CameraServer(mav_type=MAV_TYPE_CAMERA, source_component=22, camera=cam_fake1, debug=False))
        server.add_component(CameraServer(mav_type=MAV_TYPE_CAMERA, source_component=22, camera=None, debug=True))

        cam.wait_heartbeat(target_system=222, target_component=22, timeout=1)
        time.sleep(0.1)
        cam.set_target(222, 22)

        # client.component[11]._test_command(222, 22, 1)
        # for i in range (1)  :
        #     time.sleep(0.01)
        # cam.request_camera_capture_status()
        msg = cam.request_storage_information()
        print (msg)
        
        time.sleep(1)
        

INFO   | uav.MAVCom      | 18.646 |  mavcom.py:393 | Thread-59 (listen) | MAVLink Mav2: True, source_system: 111
INFO   | uav.MAVCom      | 18.748 |  mavcom.py:393 | Thread-60 (listen) | MAVLink Mav2: True, source_system: 222
INFO   | uav.CameraClien | 18.751 | component.py:111 | MainThread         | Component Started self.source_component = 11, self.mav_type = 6, self.source_system = 111
WARNIN | uav.CameraServe | 18.754 | 2975499828.py: 23 | MainThread         | Component has no camera object
INFO   | uav.CameraServe | 18.755 | component.py:111 | MainThread         | Component Started self.source_component = 22, self.mav_type = 30, self.source_system = 222
DEBUG  | uav.CameraClien | 18.756 | component.py:173 | MainThread         | Waiting for heartbeat from target_system = 222:  target_component = 22
DEBUG  | uav.CameraClien | 18.757 | component.py:181 | MainThread         | Rcvd 222/ 22 HEARTBEAT {type : 30, autopilot : 8, base_mode : 0, custom_mode : 0, system_status : 4, mavlink_v

set_mav_connection CameraClient mavcom.py:107 self.mav_connection = <MAVCom>
set_mav_connection CameraServer mavcom.py:107 self.mav_connection = <MAVCom>
None


INFO   | uav.CameraServe | 21.757 | component.py:366 | MainThread         | CameraServer closed
DEBUG  | uav.CameraServe | 21.758 | 2975499828.py:203 | MainThread         | Closed connection to camera
INFO   | uav.MAVCom      | 21.759 |  mavcom.py:441 | MainThread         | MAVCom  closed
INFO   | uav.CameraClien | 23.757 | component.py:366 | MainThread         | CameraClient closed
INFO   | uav.MAVCom      | 23.758 |  mavcom.py:441 | MainThread         | MAVCom  closed


In [ ]:
# | hide
assert False, "Stop here"

AssertionError: Stop here

In [ ]:
show_doc(CameraClient)

#### Test locally using UDP ports

#### Starting a client and server
 > on the same machine using UDP ports `14445`  with `server_system_ID=111, client_system_ID=222`

In [ ]:
from UAV.mavlink.mavcom import MAVCom
from UAV.mavlink.component import Component, mavutil
import time

MAV_TYPE_GCS = mavutil.mavlink.MAV_TYPE_GCS
MAV_TYPE_CAMERA = mavutil.mavlink.MAV_TYPE_CAMERA

class Cam1(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)

class Cam2(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)
class Cli(Component):
    def __init__(self,mav_connection, source_component, mav_type, debug=False):
        super().__init__(mav_connection=mav_connection, source_component=source_component, mav_type=mav_type,
                         debug=debug)

In [ ]:
# Test sending a command and receiving an ack from client to server
with MAVCom("udpin:localhost:14445", source_system=111, debug=False) as client:
    with MAVCom("udpout:localhost:14445", source_system=222, debug=False) as server:
        client.add_component(Cli(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
        
        for key, comp in client.component.items():
            if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                print ("*** Received heartbeat **** " )
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.component[11]._test_command(222, 22, 1)
            client.component[11]._test_command(222, 23, 1)
            
        client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")

    assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
    assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
    assert server.component[23].num_cmds_rcvd == NUM_TO_SEND

In [ ]:
#| Hide
# assert False, "Stop here"

In [ ]:
show_doc(Component.set_source_compenent)

In [ ]:
show_doc(Component.send_heartbeat)

In [ ]:
show_doc(Component.on_command_rcvd)

In [ ]:
show_doc(Component.send_command)

In [ ]:
show_doc(Component.wait_heartbeat)

In [ ]:
show_doc(Component.wait_ack)

In [ ]:
show_doc(Component.send_ping)

#### Test with Serial ports
Test using a Pixhawk connected via telemetry 2 and USB serial ports.
CamClient is set to udpin:localhost:14445 and CamServer is set to udpout:localhost:14435 udpin is so that the client can receive UDP from the mavproxy server at localhost:14445
mavproxy.py --master=/dev/ttyACM1 --baudrate 57600 --out udpout:localhost:14445 mavproxy.py --master=/dev/ttyACM3 --baudrate 57600 --out udpout:localhost:14435

In [ ]:
# Test sending a command and receiving an ack from client to server
with MAVCom("/dev/ttyACM0", source_system=111, debug=False) as client:
    with MAVCom("/dev/ttyUSB0", source_system=222, debug=False) as server:
        client.add_component(Cli(client, mav_type=MAV_TYPE_GCS, source_component = 11, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 22, debug=False))
        server.add_component(Cam1(server, mav_type=MAV_TYPE_CAMERA, source_component = 23, debug=False))
        
        for key, comp in client.component.items():
            if comp.wait_heartbeat(target_system=222, target_component=22, timeout=0.1):
                print ("*** Received heartbeat **** " )
        NUM_TO_SEND = 2
        for i in range(NUM_TO_SEND):
            client.component[11]._test_command(222, 22, 1)
            client.component[11]._test_command(222, 23, 1)
            
        client.component[11]._test_command(222, 24, 1)

    print(f"{server.source_system = };  {server.message_cnts = }")
    print(f"{client.source_system = };  {client.message_cnts = }")
    print()
    print(f"{client.source_system = } \n{client.summary()} \n")
    print(f"{server.source_system = } \n{server.summary()} \n")

    assert client.component[11].num_cmds_sent == NUM_TO_SEND * 2 + 1
    assert client.component[11].num_acks_rcvd == NUM_TO_SEND * 2
    assert client.component[11].num_acks_drop == 1
    assert server.component[22].num_cmds_rcvd == NUM_TO_SEND
    assert server.component[23].num_cmds_rcvd == NUM_TO_SEND

> For debugging help see http://localhost:3000/tutorials/mavlink_doc&debug.html and http://localhost:3000/tutorials/mavlink_doc&debug.html#debugging

In [ ]:
#| hide
# from nbdev import nbdev_export
# nbdev.nbdev_export()